# 🎨 Virtual Try-On - WORKING SOLUTION V2
### With Better Error Handling and Rate Limit Protection

This version includes:
- Better error messages
- Automatic retry logic
- Rate limit handling

## Step 1: Install Requirements

In [ ]:
!pip install -q gradio gradio_client pillow

print("✓ Installation complete!")

## Step 2: Import Libraries

In [ ]:
import gradio as gr
from gradio_client import Client, handle_file
from PIL import Image
import os
import time

print("✓ Libraries imported!")

## Step 3: Connect to IDM-VTON

In [ ]:
print("Connecting to IDM-VTON AI model...")
print("This may take a moment...")

vton_client = None
max_retries = 3

for attempt in range(max_retries):
    try:
        vton_client = Client("yisol/IDM-VTON")
        print("✓ Successfully connected to IDM-VTON!")
        print("✓ This is a REAL virtual try-on AI!")
        break
    except Exception as e:
        print(f"Attempt {attempt + 1}/{max_retries} failed: {e}")
        if attempt < max_retries - 1:
            print("Retrying in 5 seconds...")
            time.sleep(5)
        else:
            print("❌ Could not connect. The Space might be down or busy.")
            print("Please try again later.")

## Step 4: Define Try-On Function with Retry Logic

In [ ]:
# Request counter for rate limiting
request_count = 0
last_request_time = 0
MIN_REQUEST_INTERVAL = 10  # seconds between requests

def ai_tryon(body_image, clothing_image, max_retries=2):
    """
    Real AI Virtual Try-On using IDM-VTON with retry logic
    """
    global request_count, last_request_time
    
    # Check if client is connected
    if vton_client is None:
        return create_error_image("Not connected to AI service. Please restart the notebook.")
    
    # Rate limiting - add delay between requests
    current_time = time.time()
    time_since_last = current_time - last_request_time
    
    if time_since_last < MIN_REQUEST_INTERVAL:
        wait_time = MIN_REQUEST_INTERVAL - time_since_last
        print(f"⏳ Rate limiting: waiting {wait_time:.1f} seconds...")
        time.sleep(wait_time)
    
    request_count += 1
    last_request_time = time.time()
    
    print(f"\n{'='*60}")
    print(f"Try-On Request #{request_count}")
    print(f"{'='*60}")
    
    for attempt in range(max_retries):
        try:
            # Ensure images are PIL Images
            if not isinstance(body_image, Image.Image):
                body_image = Image.open(body_image).convert('RGB')
            if not isinstance(clothing_image, Image.Image):
                clothing_image = Image.open(clothing_image).convert('RGB')
            
            print(f"Body image: {body_image.size}")
            print(f"Clothing image: {clothing_image.size}")
            
            # Create temp directory
            os.makedirs('/tmp', exist_ok=True)
            
            # Save images
            body_path = '/tmp/body_temp.jpg'
            garment_path = '/tmp/garment_temp.jpg'
            
            body_image.save(body_path, 'JPEG')
            clothing_image.save(garment_path, 'JPEG')
            
            print(f"\n🤖 Calling IDM-VTON AI (Attempt {attempt + 1}/{max_retries})...")
            print("⏳ This takes 20-40 seconds...")
            
            # Call API with timeout
            result = vton_client.predict(
                dict={"background": handle_file(body_path), "layers": [], "composite": None},
                garm_img=handle_file(garment_path),
                garment_des="a clothing item",
                is_checked=True,
                is_checked_crop=False,
                denoise_steps=30,
                seed=42,
                api_name="/tryon"
            )
            
            # Process result
            if isinstance(result, tuple) and len(result) > 0:
                result_path = result[0]
                result_image = Image.open(result_path)
            else:
                result_image = Image.open(result)
            
            print("\n✅ SUCCESS!")
            print(f"{'='*60}\n")
            
            return result_image
            
        except Exception as e:
            error_str = str(e)
            print(f"\n❌ Attempt {attempt + 1} failed: {error_str}")
            
            # Check if it's a rate limit error
            if "429" in error_str or "rate limit" in error_str.lower():
                print("⚠️  Rate limit reached!")
                if attempt < max_retries - 1:
                    wait_time = 30
                    print(f"⏳ Waiting {wait_time} seconds before retry...")
                    time.sleep(wait_time)
                else:
                    return create_error_image("Rate limit reached. Please wait a minute and try again.")
            
            # Check if service is busy
            elif "503" in error_str or "busy" in error_str.lower():
                print("⚠️  AI service is busy!")
                if attempt < max_retries - 1:
                    wait_time = 15
                    print(f"⏳ Waiting {wait_time} seconds before retry...")
                    time.sleep(wait_time)
                else:
                    return create_error_image("AI service is busy. Please try again in a moment.")
            
            # Other errors
            else:
                if attempt < max_retries - 1:
                    print("⏳ Retrying in 10 seconds...")
                    time.sleep(10)
                else:
                    import traceback
                    traceback.print_exc()
                    return create_error_image(f"Error: {error_str[:100]}")
    
    return create_error_image("All retry attempts failed. Please try again later.")

def create_error_image(message):
    """
    Create an image with error message
    """
    error_img = Image.new('RGB', (768, 1024), color='#FFF3CD')
    from PIL import ImageDraw, ImageFont
    draw = ImageDraw.Draw(error_img)
    
    # Wrap text
    words = message.split(' ')
    lines = []
    current_line = []
    
    for word in words:
        current_line.append(word)
        if len(' '.join(current_line)) > 40:
            lines.append(' '.join(current_line[:-1]))
            current_line = [current_line[-1]]
    
    if current_line:
        lines.append(' '.join(current_line))
    
    # Draw text
    y_offset = 450
    for line in lines:
        bbox = draw.textbbox((0, 0), line)
        text_width = bbox[2] - bbox[0]
        x = (768 - text_width) // 2
        draw.text((x, y_offset), line, fill='#856404')
        y_offset += 30
    
    return error_img

print("✓ Try-on function with retry logic ready!")

## Step 5: Create Gradio Interface

In [ ]:
def predict(body_image, clothing_image):
    """
    Main prediction function
    """
    return ai_tryon(body_image, clothing_image)

# Create interface
iface = gr.Interface(
    fn=predict,
    inputs=[
        gr.Image(type="pil", label="📸 Body Photo"),
        gr.Image(type="pil", label="👕 Clothing Item")
    ],
    outputs=gr.Image(type="pil", label="✨ Try-On Result"),
    title="🎨 Virtual Try-On AI (v2)",
    description="""Upload a body photo and clothing. AI will dress the person with the clothing.
    
    ⚠️ IMPORTANT:
    • Processing: 20-40 seconds per try-on
    • Free tier has rate limits (5-10 requests per minute)
    • If you get rate limit error, wait 1-2 minutes
    • Automatic retry on failures
    
    💡 Tips:
    • Clear, front-facing body photos work best
    • Clothing on plain background
    • Wait 10 seconds between requests
    """,
    api_name="predict"
)

print("✓ Gradio interface created!")

## Step 6: Launch Server 🚀

In [ ]:
print("\n" + "="*60)
print("🚀 Launching server with improved error handling...")
print("="*60)

iface.launch(
    share=True,
    debug=True,
    server_port=7860,
)

print("\n" + "="*60)
print("✅ SERVER RUNNING!")
print("="*60)
print("")
print("📱 COPY THE PUBLIC URL ABOVE")
print("📱 PASTE IT INTO YOUR ANDROID APP")
print("")
print("⚠️  RATE LIMITS:")
print("   • Free tier: ~5-10 requests per minute")
print("   • If rate limited: Wait 1-2 minutes")
print("   • Automatic retry included")
print("   • Minimum 10 seconds between requests")
print("")
print("✨ Real AI - Actually dresses the person!")
print("="*60)

---

## 🎉 What's New in V2:

### ✅ Better Error Handling:
- Detects rate limit errors
- Detects busy service errors
- Shows helpful error messages

### ✅ Automatic Retry:
- Retries up to 2 times on failure
- Smart waiting times based on error type
- Rate limit: waits 30 seconds
- Busy service: waits 15 seconds

### ✅ Rate Limiting Protection:
- Enforces 10-second minimum between requests
- Prevents hitting rate limits too quickly
- Tracks request count

---

## 🔧 If You Still Get Errors:

**"Rate limit reached":**
- The free API allows ~5-10 requests per minute
- Wait 1-2 minutes before trying again
- Solution: Space out your requests

**"AI service is busy":**
- Too many people using the Space
- Wait a few minutes and retry
- Solution: Try during off-peak hours

**Consistent failures:**
- The IDM-VTON Space might be down
- Check: https://huggingface.co/spaces/yisol/IDM-VTON
- Solution: Wait for it to come back online

---